# FIN Model

In [26]:
import numpy as np
from numpy import random
import pandas as pd
from scipy.signal import cwt,ricker
from scipy import stats
from sklearn.preprocessing import StandardScaler,normalize
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.optimizers
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.models import load_model
from tensorflow.keras.layers import Flatten,concatenate,Layer,Dense,LSTM,Activation,MaxPooling2D,Dropout,Conv2D,BatchNormalization,Reshape,UpSampling2D,ZeroPadding2D

In [27]:
random.seed(222)
# Will use Mean for training, so calculate the Mean of the data
def Mean(data):
    return np.mean(data,axis=1)

def STD(data):
    return np.std(data,axis=1)

# Task 1

Train two feature imitating networks that take as input a fixed length sequence and provide as output (1) the mean and (2) the standard deviation of a fixed length input; a deep feedforward architecture should be sufficient for this.

In [28]:
# Z-scored np array
N = 100 # each group
E = 3000 # length of the data groups
input_arr = np.random.randint(10, size=(N,1, E) )
scaled_arr = np.zeros(input_arr.shape)

#print(input_arr)

for ii in range(input_arr.shape[1]):
#     scaler = StandardScaler()
#     scaled_arr[:,ii,:] = scaler.fit_transform(input_arr[:,ii,:])
    scaled_arr[:,ii,:] = input_arr[:,ii,:]
    
#print(scaled_arr.shape)
#print(scaled_arr)

# move the axis of the matrix as as # Epochs X T X Ch

scaled_arr = np.moveaxis(scaled_arr,2,0)
print(scaled_arr.shape)

do_cwt = False
if do_cwt:
    # Continuous wavelet transform
    flat_data = np.squeeze(scaled_arr)
    scaled_arr_cwt = []
    for ii in range(flat_data.shape[0]):
        widths = np.arange(1, 31)
        cwtmatr = cwt(flat_data[ii], ricker, widths)
        scaled_arr_cwt.append(np.moveaxis(cwtmatr,1,0))
    scaled_arr3d = np.stack(scaled_arr_cwt)
else:
    scaled_arr3d = scaled_arr
    
print(scaled_arr)



(3000, 100, 1)
[[[6.]
  [6.]
  [7.]
  ...
  [8.]
  [0.]
  [2.]]

 [[3.]
  [9.]
  [4.]
  ...
  [6.]
  [1.]
  [9.]]

 [[2.]
  [6.]
  [2.]
  ...
  [3.]
  [4.]
  [6.]]

 ...

 [[2.]
  [5.]
  [9.]
  ...
  [6.]
  [5.]
  [4.]]

 [[8.]
  [1.]
  [9.]
  ...
  [3.]
  [7.]
  [0.]]

 [[4.]
  [2.]
  [0.]
  ...
  [5.]
  [2.]
  [5.]]]


In [29]:

#squeeze the data in
feature_res = np.squeeze(Mean(scaled_arr))
feature_res_std = np.squeeze(STD(scaled_arr))

# the predict result should be
# print(np.mean(scaled_arr))
# print(feature_res)
# print(feature_res.shape)
print(feature_res_std.shape)

(3000,)


In [30]:
# Make FIN
def make_FIN(topology,size3D,N,name="my-FIN"):
    input_layer = keras.Input(shape=(N,size3D))
    x = Flatten()(input_layer)
    for layer in topology['layers']:
        x = Dense(layer['output_shape'], activation='relu')(x)
        if layer['BatchNorm']:
            x = BatchNormalization(axis=1)(x)
    fin_out = Dense(1, activation='linear')(x)
    fin = keras.Model(input_layer, fin_out, name=name)
    return fin

topology = np.load('RandomTopology.npy', allow_pickle=True)[1]


fin = make_FIN(topology,1,N)
fin2 = make_FIN(topology,1,N)

print(scaled_arr3d.shape)
print(feature_res.shape)


(3000, 100, 1)
(3000,)


In [31]:
learning_rate=topology['lr']
#learning_rate= 0.0005
# learning_rate= 0.00001
momentum=topology['momentum']
nesterov=topology['nesterov']
b1 = topology['b1']
b2 = topology['b2']
if topology['opt'] == 'SGD':
    grad_desc_algorithm = tf.keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum,nesterov=nesterov)
if topology['opt'] == 'ADAM':
    grad_desc_algorithm = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=b1,beta_2=b2)
    
#es = EarlyStopping(monitor='accuracy', mode='min', verbose=1,patience=3)

fin.compile(optimizer = grad_desc_algorithm,loss = 'mean_squared_error')
fin2.compile(optimizer = grad_desc_algorithm,loss = 'mean_squared_error')



# loss_d = fin.fit(scaled_arr3d, feature_res, 
#             epochs = 1000,
#             batch_size = 500,
#             verbose   = 1)    

# loss_d2 = fin2.fit(scaled_arr3d, feature_res_std, 
#             epochs = 1000,
#             batch_size = 500,
#             verbose   = 1)   


In [32]:
Predirect_arr = np.random.randint(10, size=(1,100, 1)).T

# fin.predict(Predirect_arr)
# print(np.mean(Predirect_arr))
# print(fin.predict(Predirect_arr))
# print(np.std(Predirect_arr))
# print(fin2.predict(Predirect_arr))


In [33]:
#fin.save("./Models/Modelstask1Mean.h5")
# fin2.save("./Models/Modelstask1Std.h5")

In [34]:
task1MeanModel = load_model('./Models/Modelstask1Mean.h5')


In [37]:
Predirect_arr = np.random.randint(10, size=(1,100, 1)).T
print(np.mean(Predirect_arr))
task1MeanModel.predict(Predirect_arr)


4.84


array([[4.859808]], dtype=float32)

# TASK 2

Train two feature imitating networks that take as input an arbitrary length sequence and provide as output (1) the mean and (2) the standard deviation of the sequence; a LSTM, or other recurrent architecture should be sufficient for this.

In [138]:
# Generate random length of array
# T x Ch x Epochs
row = random.randint(20, 50) # Arbitrary length of data
column = 25 # data per group
input_arr = np.random.randint(10, size=(row,column, 1) )

print(input_arr.shape)

(45, 25, 1)


In [139]:
# Target Arr one for Mean, one for Std
target_arr = np.squeeze(Mean(input_arr))
target_arr2 = np.squeeze(STD(input_arr))
print(target_arr.shape)

# the first model for calculate the mean, second model for std, 
# also picked different preset parameters of topology for two models
model = make_FIN(np.load('RandomTopology.npy', allow_pickle=True)[2],1,column)
model2 = make_FIN(np.load('RandomTopology.npy', allow_pickle=True)[3],1,column)

# Having the code setting up the parameters here for changes

#learning_rate=topology['lr']
#learning_rate= 0.00001
# momentum=topology['momentum']
# nesterov=topology['nesterov']
# b1 = topology['b1']
# b2 = topology['b2']
# if topology['opt'] == 'SGD':
#     grad_desc_algorithm = tf.keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum,nesterov=nesterov)
# if topology['opt'] == 'ADAM':
#     grad_desc_algorithm = tf.keras.optimizers.Adam(learning_rate=learning_rate,beta_1=b1,beta_2=b2)
    
#make LSTM model, make all the nodes sequential first for LSTM reccursive
model = Sequential()
model2 = Sequential()


model.add(LSTM(1,batch_input_shape=(None,column,1 ), activation='relu'))
#model.add(Dense(4,activation='sigmoid'))

model2.add(LSTM(1,batch_input_shape=(None,column,1 ), activation='relu'))

(45,)


In [140]:
# note: use SGD gonna be a lot less accuracy than adam, from testing results
# gonna use adam right now if not fit with Fin's feature will change later

#model.compile(loss ='mean_squared_error', optimizer='adam')
#model2.compile(loss ='mean_squared_error', optimizer='adam')

In [141]:
#model.summary()

In [142]:
# In real training I used different aprecciated epochs for them until the loss signficently not decreasing more.

# model.fit(input_arr, target_arr, 
#             epochs = 18000,
#             verbose   = 1)
# model2.fit(input_arr, target_arr2, 
#             epochs = 9000,
#             verbose   = 1)

In [205]:
pp_arr = np.random.randint(10, size=(1,25, 1))

# model = load_model('./Models/Modelstask2Mean.h5')
# model2 = load_model('./Models/Modelstask2Std.h5')
print(model.predict(pp_arr))
print(np.mean(pp_arr))

print(model2.predict(pp_arr))
print(np.std(pp_arr))



[[3.5917158]]
3.6
[[2.8389986]]
2.814249455894058


In [119]:
# save the mean model for task2
#model.save("./Models/Modelstask2Mean.h5")
#model2.save("./Models/Modelstask2Std.h5")

# TASK 3

Train one feature imitating network that takes as input an arbitrary length sequence, and provides both the mean and standard deviation of that sequence; an LSTM or other recurrent architecture with multiple outputs should be sufficient for this task.

In [191]:
# Will use the same input data as Task 2 for Task 3

model_task3 = make_FIN(np.load('RandomTopology.npy', allow_pickle=True)[4],1,column)

In [192]:
model_task3 = Sequential()
model_task3.add(LSTM(2,batch_input_shape=(None,column,1 ), activation='relu'))
model_task3.compile(loss ='mean_squared_error', optimizer='adam')

In [193]:
feature_output_arr = np.stack([target_arr,target_arr2], axis=1)
model_task3.fit(input_arr, feature_output_arr, 
            epochs = 9000,
            verbose   = 1)

Epoch 1/9000
2/2 [==============================] - 1s 106ms/step - loss: 3134.4394
Epoch 2/9000
2/2 [==============================] - 0s 107ms/step - loss: 2746.4534
Epoch 3/9000
2/2 [==============================] - 0s 172ms/step - loss: 2503.3612
Epoch 4/9000
2/2 [==============================] - 0s 157ms/step - loss: 2318.9998
Epoch 5/9000
2/2 [==============================] - 0s 117ms/step - loss: 2062.9867
Epoch 6/9000
2/2 [==============================] - 0s 135ms/step - loss: 1935.0085
Epoch 7/9000
2/2 [==============================] - 0s 160ms/step - loss: 1882.6418
Epoch 8/9000
2/2 [==============================] - 0s 59ms/step - loss: 1638.7000
Epoch 9/9000
2/2 [==============================] - 0s 132ms/step - loss: 1598.7762
Epoch 10/9000
2/2 [==============================] - 0s 98ms/step - loss: 1528.3493
Epoch 11/9000
2/2 [==============================] - 0s 88ms/step - loss: 1393.3805
Epoch 12/9000
2/2 [==============================] - 0s 84ms/step - loss: 131

2/2 [==============================] - 0s 108ms/step - loss: 173.9463
Epoch 100/9000
2/2 [==============================] - 0s 68ms/step - loss: 170.7864
Epoch 101/9000
2/2 [==============================] - 0s 62ms/step - loss: 156.1776
Epoch 102/9000
2/2 [==============================] - 0s 87ms/step - loss: 167.1367
Epoch 103/9000
2/2 [==============================] - 0s 77ms/step - loss: 160.3322
Epoch 104/9000
2/2 [==============================] - 0s 85ms/step - loss: 162.0384
Epoch 105/9000
2/2 [==============================] - 0s 69ms/step - loss: 148.4584
Epoch 106/9000
2/2 [==============================] - 0s 76ms/step - loss: 144.5158
Epoch 107/9000
2/2 [==============================] - 0s 92ms/step - loss: 144.8243
Epoch 108/9000
2/2 [==============================] - 0s 71ms/step - loss: 141.2055
Epoch 109/9000
2/2 [==============================] - 0s 113ms/step - loss: 140.2050
Epoch 110/9000
2/2 [==============================] - 0s 79ms/step - loss: 132.5983
Epoch

2/2 [==============================] - 0s 87ms/step - loss: 28.3707
Epoch 198/9000
2/2 [==============================] - 0s 52ms/step - loss: 27.9742
Epoch 199/9000
2/2 [==============================] - 0s 58ms/step - loss: 24.8821
Epoch 200/9000
2/2 [==============================] - 0s 58ms/step - loss: 25.5446
Epoch 201/9000
2/2 [==============================] - 0s 55ms/step - loss: 26.1535
Epoch 202/9000
2/2 [==============================] - 0s 77ms/step - loss: 26.2094
Epoch 203/9000
2/2 [==============================] - 0s 66ms/step - loss: 26.7455
Epoch 204/9000
2/2 [==============================] - 0s 60ms/step - loss: 26.9312
Epoch 205/9000
2/2 [==============================] - 0s 76ms/step - loss: 27.1288
Epoch 206/9000
2/2 [==============================] - 0s 66ms/step - loss: 25.9201
Epoch 207/9000
2/2 [==============================] - 0s 65ms/step - loss: 26.0246
Epoch 208/9000
2/2 [==============================] - 0s 62ms/step - loss: 24.3866
Epoch 209/9000
2/2 

2/2 [==============================] - 0s 86ms/step - loss: 18.1554
Epoch 296/9000
2/2 [==============================] - 0s 70ms/step - loss: 18.8226
Epoch 297/9000
2/2 [==============================] - 0s 79ms/step - loss: 18.0596
Epoch 298/9000
2/2 [==============================] - 0s 93ms/step - loss: 18.5045
Epoch 299/9000
2/2 [==============================] - 0s 73ms/step - loss: 18.3608
Epoch 300/9000
2/2 [==============================] - 0s 76ms/step - loss: 18.3251
Epoch 301/9000
2/2 [==============================] - 0s 74ms/step - loss: 18.2184
Epoch 302/9000
2/2 [==============================] - 0s 99ms/step - loss: 18.6108
Epoch 303/9000
2/2 [==============================] - 0s 83ms/step - loss: 18.2050
Epoch 304/9000
2/2 [==============================] - 0s 99ms/step - loss: 17.0712
Epoch 305/9000
2/2 [==============================] - 0s 52ms/step - loss: 18.2925
Epoch 306/9000
2/2 [==============================] - 0s 54ms/step - loss: 17.7949
Epoch 307/9000
2/2 

2/2 [==============================] - 0s 63ms/step - loss: 15.5370
Epoch 394/9000
2/2 [==============================] - 0s 62ms/step - loss: 15.2372
Epoch 395/9000
2/2 [==============================] - 0s 60ms/step - loss: 15.1592
Epoch 396/9000
2/2 [==============================] - 0s 47ms/step - loss: 15.0889
Epoch 397/9000
2/2 [==============================] - 0s 46ms/step - loss: 15.4676
Epoch 398/9000
2/2 [==============================] - 0s 47ms/step - loss: 15.3335
Epoch 399/9000
2/2 [==============================] - 0s 46ms/step - loss: 15.6900
Epoch 400/9000
2/2 [==============================] - 0s 46ms/step - loss: 15.1542
Epoch 401/9000
2/2 [==============================] - 0s 47ms/step - loss: 15.3383
Epoch 402/9000
2/2 [==============================] - 0s 61ms/step - loss: 15.2380
Epoch 403/9000
2/2 [==============================] - 0s 62ms/step - loss: 15.6161
Epoch 404/9000
2/2 [==============================] - 0s 80ms/step - loss: 15.9465
Epoch 405/9000
2/2 

...

2/2 [==============================] - 0s 32ms/step - loss: 0.0290
Epoch 8533/9000
2/2 [==============================] - 0s 39ms/step - loss: 0.0300
Epoch 8534/9000
2/2 [==============================] - 0s 44ms/step - loss: 0.0274
Epoch 8535/9000
2/2 [==============================] - 0s 49ms/step - loss: 0.0278
Epoch 8536/9000
2/2 [==============================] - 0s 55ms/step - loss: 0.0299
Epoch 8537/9000
2/2 [==============================] - 0s 52ms/step - loss: 0.0288
Epoch 8538/9000
2/2 [==============================] - 0s 56ms/step - loss: 0.0298
Epoch 8539/9000
2/2 [==============================] - 0s 66ms/step - loss: 0.0279
Epoch 8540/9000
2/2 [==============================] - 0s 62ms/step - loss: 0.0279
Epoch 8541/9000
2/2 [==============================] - 0s 62ms/step - loss: 0.0281
Epoch 8542/9000
2/2 [==============================] - 0s 54ms/step - loss: 0.0293
Epoch 8543/9000
2/2 [==============================] - 0s 57ms/step - loss: 0.0294
Epoch 8544/9000
2/2 

2/2 [==============================] - 0s 58ms/step - loss: 0.0267
Epoch 8631/9000
2/2 [==============================] - 0s 88ms/step - loss: 0.0300
Epoch 8632/9000
2/2 [==============================] - 0s 56ms/step - loss: 0.0279
Epoch 8633/9000
2/2 [==============================] - 0s 71ms/step - loss: 0.0264
Epoch 8634/9000
2/2 [==============================] - 0s 69ms/step - loss: 0.0277
Epoch 8635/9000
2/2 [==============================] - 0s 121ms/step - loss: 0.0299
Epoch 8636/9000
2/2 [==============================] - 0s 69ms/step - loss: 0.0275
Epoch 8637/9000
2/2 [==============================] - 0s 67ms/step - loss: 0.0280
Epoch 8638/9000
2/2 [==============================] - 0s 71ms/step - loss: 0.0263
Epoch 8639/9000
2/2 [==============================] - 0s 76ms/step - loss: 0.0284
Epoch 8640/9000
2/2 [==============================] - 0s 49ms/step - loss: 0.0286
Epoch 8641/9000
2/2 [==============================] - 0s 66ms/step - loss: 0.0287
Epoch 8642/9000
2/2

2/2 [==============================] - 0s 70ms/step - loss: 0.0288
Epoch 8729/9000
2/2 [==============================] - 0s 64ms/step - loss: 0.0292
Epoch 8730/9000
2/2 [==============================] - 0s 113ms/step - loss: 0.0251
Epoch 8731/9000
2/2 [==============================] - 0s 62ms/step - loss: 0.0284
Epoch 8732/9000
2/2 [==============================] - 0s 51ms/step - loss: 0.0283
Epoch 8733/9000
2/2 [==============================] - 0s 77ms/step - loss: 0.0277
Epoch 8734/9000
2/2 [==============================] - 0s 108ms/step - loss: 0.0282
Epoch 8735/9000
2/2 [==============================] - 0s 67ms/step - loss: 0.0275
Epoch 8736/9000
2/2 [==============================] - 0s 61ms/step - loss: 0.0289
Epoch 8737/9000
2/2 [==============================] - 0s 63ms/step - loss: 0.0277
Epoch 8738/9000
2/2 [==============================] - 0s 60ms/step - loss: 0.0293
Epoch 8739/9000
2/2 [==============================] - 0s 58ms/step - loss: 0.0292
Epoch 8740/9000
2/

2/2 [==============================] - 0s 59ms/step - loss: 0.0290
Epoch 8827/9000
2/2 [==============================] - 0s 57ms/step - loss: 0.0274
Epoch 8828/9000
2/2 [==============================] - 0s 71ms/step - loss: 0.0295
Epoch 8829/9000
2/2 [==============================] - 0s 67ms/step - loss: 0.0285
Epoch 8830/9000
2/2 [==============================] - 0s 70ms/step - loss: 0.0291
Epoch 8831/9000
2/2 [==============================] - 0s 68ms/step - loss: 0.0286
Epoch 8832/9000
2/2 [==============================] - 0s 49ms/step - loss: 0.0271
Epoch 8833/9000
2/2 [==============================] - 0s 49ms/step - loss: 0.0294
Epoch 8834/9000
2/2 [==============================] - 0s 47ms/step - loss: 0.0282
Epoch 8835/9000
2/2 [==============================] - 0s 49ms/step - loss: 0.0295
Epoch 8836/9000
2/2 [==============================] - 0s 51ms/step - loss: 0.0264
Epoch 8837/9000
2/2 [==============================] - 0s 49ms/step - loss: 0.0296
Epoch 8838/9000
2/2 

2/2 [==============================] - 0s 51ms/step - loss: 0.0287
Epoch 8925/9000
2/2 [==============================] - 0s 79ms/step - loss: 0.0256
Epoch 8926/9000
2/2 [==============================] - 0s 60ms/step - loss: 0.0271
Epoch 8927/9000
2/2 [==============================] - 0s 44ms/step - loss: 0.0273
Epoch 8928/9000
2/2 [==============================] - 0s 65ms/step - loss: 0.0296
Epoch 8929/9000
2/2 [==============================] - 0s 45ms/step - loss: 0.0251
Epoch 8930/9000
2/2 [==============================] - 0s 34ms/step - loss: 0.0288
Epoch 8931/9000
2/2 [==============================] - 0s 43ms/step - loss: 0.0280
Epoch 8932/9000
2/2 [==============================] - 0s 100ms/step - loss: 0.0256
Epoch 8933/9000
2/2 [==============================] - 0s 67ms/step - loss: 0.0291
Epoch 8934/9000
2/2 [==============================] - 0s 62ms/step - loss: 0.0272
Epoch 8935/9000
2/2 [==============================] - 0s 74ms/step - loss: 0.0275
Epoch 8936/9000
2/2

In [206]:
model_task3.predict(pp_arr)
# could compare with task 2 result after loading the model

array([[3.508058 , 2.6829824]], dtype=float32)

In [207]:
#save the model
model_task3.save("./Models/Modelstask3.h5")